In [9]:
# This cell imports stuff, and sets up a bot instance etc

import sys
import lzma
from s2clientprotocol.sc2api_pb2 import Response, ResponseObservation
from MapAnalyzer.MapData import MapData
from MapAnalyzer.constructs import (ChokeArea, MDRamp, RawChoke,
                                    VisionBlockerArea)
import pickle
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../src")
sys.path.append("../src/ares")
from sc2.position import Point2
from sc2.client import Client
from sc2.game_data import GameData
from sc2.game_info import GameInfo
from sc2.game_state import GameState
from unittest.mock import patch
from ares import AresBot
from ares.dicts.unit_data import UNIT_DATA
from sc2.bot_ai import BotAI
from typing import Optional, Union

async def build_bot_object_from_pickle_data(raw_game_data, raw_game_info, raw_observation) -> AresBot:
    # Build fresh bot object, and load the pickled data into the bot object
    bot = BotAI()
    game_data = GameData(raw_game_data.data)
    game_info = GameInfo(raw_game_info.game_info)
    game_state = GameState(raw_observation)
    bot._initialize_variables()
    client = Client(True)
    
    bot._prepare_start(client=client, player_id=1, game_info=game_info, game_data=game_data)
    with patch.object(Client, "query_available_abilities_with_tag", return_value={}):
        await bot._prepare_step(state=game_state, proto_game_info=raw_game_info)
        bot._prepare_first_step()
        # await bot.register_managers()
    return bot

BERLINGRAD = "../tests/pickle_data/BerlingradAIE.xz"
with lzma.open(BERLINGRAD, "rb") as f:
    raw_game_data, raw_game_info, raw_observation = pickle.load(f)

# initiate a BotAI and MapAnalyzer instance
bot = await build_bot_object_from_pickle_data(raw_game_data, raw_game_info, raw_observation)
data = MapData(bot)

# common variables
grid = data.get_pyastar_grid()
position = bot.enemy_start_locations[0]
units = bot.all_units

%load_ext line_profiler
%load_ext Cython

2023-06-26 19:37:37.022 | INFO     | MapAnalyzer.MapData:__init__:122 - dev Compiling Berlingrad AIE 
 Version dev Map Compilation Progress : 0.4it [00:00,  1.68it/s]

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [25]:
target = bot.game_info.map_center
chokes = [
    ch
    for ch in data.in_region_p(target).region_chokes
    if type(ch) != VisionBlockerArea
]
raw_choke = chokes[-1]
assert type(raw_choke) == RawChoke

In [31]:
raw_choke.side_a

(92, 67)

# Finding siege points

## Get line between points

We need this function in the final find siege function

### python

In [10]:
def get_line_between_points(pa, pb) -> list[float]:
    """
    Given points a and b, return the line in the form Ax + By = C.
    Returns [A, B, C]
    """
    x1, y1 = pa
    x2, y2 = pb
    if x1 == x2:
        return [1, 0, x1]
    else:
        slope = (y2 - y1) / (x2 - x1)
        return [-slope, 1, y1 - (slope * x1)]

In [28]:
%timeit choke_a, choke_b, choke_c = get_line_between_points(raw_choke.side_a, raw_choke.side_b)

390 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### cython

In [37]:
%%cython

cpdef list cy_get_line_between_points((unsigned int, unsigned int) pa, (unsigned int, unsigned int) pb):
    """
    Given points a and b, return the line in the form Ax + By = C.
    Returns [A, B, C]
    """
    cdef:
        unsigned int x1, y1, x2, y2
        double slope

    x1, y1 = pa
    x2, y2 = pb
    if x1 == x2:
        return [1, 0, x1]
    else:
        slope = (y2 - y1) / (x2 - x1)
        return [-slope, 1, y1 - (slope * x1)]

In [38]:
%timeit choke_a, choke_b, choke_c = cy_get_line_between_points(raw_choke.side_a, raw_choke.side_b)

134 ns ± 0.0732 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [18]:
def get_siege_chokes(
    target: Point2,
    chokes: Optional[
        tuple[Union[ChokeArea, MDRamp, RawChoke, VisionBlockerArea]]
    ] = None,
    max_distance: int = 30,
    min_setup_area: int = 0,
) -> tuple[
    dict[Point2, Union[MDRamp, RawChoke, VisionBlockerArea]],
    dict[Point2, Union[MDRamp, RawChoke, VisionBlockerArea]],
]:
    """
    Find the chokes for sieging a position. Criteria are:
        - The side of the choke closest to the position must not be higher ground
        - There must be at least min_setup_area points on our side of the choke
        TODO: add more stuff
    @param target: the point to siege
    @param chokes: a list of chokes we want to check, if we know them
    @param max_distance: how far away from the point the near side of the choke is allowed to be
    @param min_setup_area: minimum number of valid points on the near side of the choke
    @return: A dictionary with the side we want to go to as the key and the ChokeArea as the value
    """
    raw_pathfind = self.manager_mediator.find_raw_path
    grid = self.manager_mediator.get_cached_ground_grid
    # if we're not given chokes (such as the point isn't a base that we calculated earlier), find them
    if not chokes:
        chokes = [
            ch
            for ch in self.map_data.in_region_p(target).region_chokes
            if type(ch) != VisionBlockerArea
        ]
    chokes_points: List[Set[Point2]] = [ch.points for ch in chokes]
    # go through the chokes and check heights of both sides so we can see if we'll have the high ground
    high_or_even_ground_chokes: Dict[
        Point2, Union[MDRamp, RawChoke, VisionBlockerArea]
    ] = {}
    low_ground_or_small_chokes: Dict[
        Point2, Union[MDRamp, RawChoke, VisionBlockerArea]
    ] = {}
    for ch in chokes:
        if type(ch) == RawChoke:
            # get the line between sides of the choke
            choke_a, choke_b, choke_c = get_line_between_points(
                ch.side_a, ch.side_b
            )
            # the A value of the line bisecting Ax + By = C  is -B/A
            if choke_a == 0:
                # horizontal line, the value is 1 (can't divide by 0)
                bisect_a = 1
            else:
                bisect_a = -choke_b / choke_a
            # get the intersection of the two lines (the intersection is the midpoint of the choke line)
            if choke_a == 0:
                # the choke makes a horizontal line, so we want the average of the x values
                intersect_x = (ch.side_b.x + ch.side_a.x) / 2
                intersect_y = ch.side_a.y
            elif choke_b == 0:
                # the choke makes a vertical line, so we want the average of the y values
                intersect_x = ch.side_a.x
                intersect_y = (ch.side_b.y + ch.side_a.y) / 2
            else:
                # the line is neither horizontal nor vertical, use the x midpoint to find y
                intersect_x = (ch.side_b.x + ch.side_a.x) / 2
                intersect_y = choke_c - choke_a * intersect_x
            # find a point on either side of the choke line to check heights for
            raw_a = translate_point_along_line(
                (intersect_x, intersect_y), bisect_a, 4
            )
            raw_b = translate_point_along_line(
                (intersect_x, intersect_y), bisect_a, -4
            )
            point_a = Point2(raw_a).rounded
            point_b = Point2(raw_b).rounded
        elif type(ch) == MDRamp:
            point_a = ch.top_center.towards(ch.bottom_center, -2).rounded
            point_b = ch.bottom_center.towards(ch.top_center, -2).rounded
        else:
            continue
        # get the heights on either side of the choke
        height_a = self.ai.get_terrain_height(point_a)
        height_b = self.ai.get_terrain_height(point_b)

        # find the pathing distances or skip this choke since something's wrong
        # also skip if either side goes through more than one choke
        # noinspection PyProtectedMember
        path_a = raw_pathfind(
            start=point_a,
            target=target,
            grid=grid,
            sensitivity=1,
        )
        if path_a:
            if number_of_chokes_pathed_through(path_a, chokes_points) > 1:
                continue
            dist_a = len(path_a)
        else:
            continue
        # noinspection PyProtectedMember
        path_b = raw_pathfind(
            start=point_b,
            target=target,
            grid=grid,
            sensitivity=1,
        )
        if path_b:
            if number_of_chokes_pathed_through(path_b, chokes_points) > 1:
                continue
            dist_b = len(path_b)
        else:
            continue

        """
        Siege points should be:
            - the same height or higher than the target
            - within the maximum distance
            - within a large enough area
            - only path through one choke to get to the target
        """
        if height_a < height_b:
            # side_a is lower
            if dist_a < dist_b and dist_a < max_distance:
                if self.get_flood_fill_area(point_b)[0] >= min_setup_area:
                    high_or_even_ground_chokes[point_b] = ch
            else:
                low_ground_or_small_chokes[point_b] = ch
        elif height_b < height_a:
            # side_b is lower
            if dist_b < dist_a and dist_b < max_distance:
                if self.get_flood_fill_area(point_a)[0] >= min_setup_area:
                    high_or_even_ground_chokes[point_a] = ch
            else:
                low_ground_or_small_chokes[point_a] = ch
        else:
            # the chokes are of equal height, give the side further from the target if it's large enough
            # if they're somehow the same height and same distance away, skip for now
            # TODO: find some way to handle this if we want- it shouldn't happen much though
            if dist_a < dist_b and dist_a < max_distance:
                if self.get_flood_fill_area(point_b)[0] >= min_setup_area:
                    high_or_even_ground_chokes[point_b] = ch
                else:
                    low_ground_or_small_chokes[point_b] = ch
            elif dist_b < dist_a and dist_b < max_distance:
                if self.get_flood_fill_area(point_a)[0] >= min_setup_area:
                    high_or_even_ground_chokes[point_a] = ch
                else:
                    low_ground_or_small_chokes[point_a] = ch
    return high_or_even_ground_chokes, low_ground_or_small_chokes

In [17]:
get_siege_chokes(bot.game_info.map_center)

NameError: name 'self' is not defined